In [2]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_2.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    max_index = np.argmax(cross_corrs)
    best_tau = max_index - max_tau  # Convert array index back to tau value

    # Apply threshold check and adjust max_corr based on tau direction
    if max_corr >= threshold:
        return max_corr if best_tau >= 0 else -max_corr
    else:
        return 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename, max_tau):
    submatrix = pd.DataFrame(index=columns, columns=columns)

    # Loop through column pairs, skipping redundant calculations
    for col1, col2 in product(columns, repeat=2):
        if col1 <= col2:  # Ensures each pair is calculated only once
            correlation = time_delayed_cross_correlation(df, col1, col2, max_tau)
            submatrix.loc[col1, col2] = correlation
            submatrix.loc[col2, col1] = correlation  # Fill symmetric position

    submatrix.to_csv(filename)
    print(f"Submatrix {filename} has been saved.")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix{filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


#filename_sub = "2_sub_mat_3.csv"
#compute_submatrix(data, group_3, filename_sub, max_tau)

filename_cross = "1_cross_matrix_2_3.csv"
compute_cross_group_matrix(data, group_2, group_3, filename_cross)


submatrix1_cross_matrix_2_3.csv has been saved


,38099.0,38101.0,38103.0,38105.0,39001.0,39003.0,39005.0,39007.0,39009.0,39011.0,...,56027.0,56029.0,56031.0,56033.0,56035.0,56037.0,56039.0,56041.0,56043.0,56045.0
21083.0,-0.417001,0.387399,0.262845,0.594676,0.377752,0.539311,0.392534,0.136347,0.515778,0.529651,...,0,-0.535006,-0.281437,-0.383048,0.399564,-0.539343,-0.496919,-0.291169,0.204202,0.294618
21085.0,-0.080972,-0.236283,0.138432,-0.221829,-0.102638,-0.231527,-0.11948,0.316433,0,-0.236241,...,0,-0.142745,-0.096082,-0.154041,-0.171658,-0.157888,-0.275258,0,0.172461,0.1022
21087.0,0.352814,0.359655,-0.308593,0.491078,-0.414905,0.446528,-0.312915,0,-0.267456,0.378866,...,0,-0.466792,0.320657,-0.690362,-0.338381,-0.386048,-0.600727,-0.255194,0.442421,0.490301
21089.0,-0.318704,-0.423925,-0.361159,-0.585951,0.494889,0.638817,-0.454251,0,-0.451844,-0.549803,...,0,-0.36906,0.219564,-0.49249,-0.423788,-0.510208,-0.559035,-0.303842,0.338179,0.331626
21091.0,0.420441,0.41776,0.267179,0.452355,0.345844,0.424787,0.543322,-0.077441,0.51657,0.437978,...,0,-0.525143,-0.316731,0.330517,0.428971,-0.436181,0.575122,-0.245914,0.208126,0.354737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38089.0,-0.370939,-0.406003,-0.391273,-0.562796,-0.481582,0.583009,-0.366499,0,-0.357854,0.562481,...,0,-0.323169,0.253663,-0.640284,-0.441663,-0.429466,-0.649584,-0.1254,0.458858,0.43898
38091.0,-0.513737,-0.330415,0.380895,-0.411867,0.452883,0.403981,-0.530565,0.053611,-0.47647,0.397704,...,0,0.340656,-0.243638,0.536562,-0.548509,-0.468764,-0.421806,-0.220902,0.329366,-0.278417
38093.0,-0.418718,0.293566,0.329376,-0.333077,0.27022,0.238936,-0.252591,-0.287686,-0.42264,0.305292,...,0,0.170575,0.301661,0.366285,-0.266967,-0.241395,-0.350871,0.295983,0.290192,-0.263541
38095.0,-0.524525,0.291465,0.175976,0.312579,0.286795,0.224336,0.386547,0.096758,0.371525,0.411731,...,0,-0.362624,-0.381218,0.426484,0.337483,0.388272,0.404686,0.1589,0.231006,-0.352808
